In [1]:
# backend/aiml_models/agent_teams/agent_tailored_cover_letter/src/notebook/playground.ipynb
import sys
import os

# Locate the "src" dynamically from notebook location
NOTEBOOK_DIR = os.path.dirname(os.path.abspath("__file__"))
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, ".."))

SRC_DIR = os.path.join(PROJECT_ROOT, "src")

# Debug prints to check
print(f"Current Notebook Dir: {NOTEBOOK_DIR}")
print(f"Project Root (agent_tailored_cover_letter): {PROJECT_ROOT}")
print(f"Adding SRC_DIR to sys.path: {SRC_DIR}")

sys.path.append(SRC_DIR)




Current Notebook Dir: /home/mangabat/projects/portofolio/backend/aiml_models/agent_teams/agent_tailored_cover_letter
Project Root (agent_tailored_cover_letter): /home/mangabat/projects/portofolio/backend/aiml_models/agent_teams
Adding SRC_DIR to sys.path: /home/mangabat/projects/portofolio/backend/aiml_models/agent_teams/src


In [2]:
# Section 1
# File: backend/aiml_models/agent_teams/agent_tailored_cover_letter/src/infrastructure/corrections_client.py

import httpx
from typing import List, Literal
from src.config.config_top_level import ConfigTopLevel

class CorrectionsClient:
    def __init__(self) -> None:
        self.config = ConfigTopLevel.load_config()
        self.base_url = "http://localhost:8010/corrections"

    def fetch_corrections(self, correction_type: Literal["word", "sentence", "skill"]) -> List[str]:
        response = httpx.get(self.base_url, params={"correction_type": correction_type})
        response.raise_for_status()
        return response.json()  # This gives you a proper list of dicts

corrections_client = CorrectionsClient()
skills_response  = corrections_client.fetch_corrections("skill")
skillsets = [skill["text"] for skill in skills_response]


for i in skillsets:
    print(i)


ConnectError: [Errno 111] Connection refused

In [ ]:
# Section 2
# File: backend/aiml_models/agent_teams/agent_tailored_cover_letter/src/core/company_analysis/components/analysis_response_parser.py
import json
from langchain_core.output_parsers import PydanticOutputParser
from src.core.data_models.analysis_result_model import JobAnalysisResult

class JobAnalysisResultParser:
    """
    Purpose:
        Parses the raw LLM response into a structured JobAnalysisResult object.

    Capabilities:
        - Converts raw LLM output (usually a JSON string) into a JobAnalysisResult.
        - Uses PydanticOutputParser to leverage type enforcement.

    Reasoning:
        - Keeping parsing logic separate from data models ensures:
        ✅ Parsing can evolve (different LLM formats) without changing data contracts.
        ✅ Data models stay clean (no LLM-specific logic).
    """
    def __init__(self) -> None:
        self.parser = PydanticOutputParser(pydantic_object=JobAnalysisResult)


    def parse(self, llm_response: str) -> JobAnalysisResult:
        """
        Converts raw string response from LLM into structured JobAnalysisResult.

        Args:
            llm_response: Raw JSON string from LLM.

        Returns:
            JobAnalysisResult: Parsed and validated structured result.
        """
        return self.parser.parse(llm_response)

# Instantiate the parser
response_parser = JobAnalysisResultParser()

# Sample valid JSON string (simulate what an LLM would return)
mock_llm_response = """
{
    "company_name": "AwesomeTech",
    "job_title": "Data Scientist",
    "analysis_output": "The position requires advanced data analysis and machine learning skills.",
    "employees_skills_requirement": {
        "Python": true,
        "Machine Learning": true,
        "PowerBI": false
    },
    "matching_skills": {
        "Python": true,
        "Machine Learning": true
    }
}
"""

# Parse and inspect result
parsed_result = response_parser.parse(mock_llm_response)

print("\n✅ Parsed Job Analysis Result:")
print(json.dumps(parsed_result.model_dump(), indent=2))



✅ Parsed Job Analysis Result:
{
  "company_name": "AwesomeTech",
  "job_title": "Data Scientist",
  "analysis_output": "The position requires advanced data analysis and machine learning skills.",
  "employees_skills_requirement": {
    "Python": true,
    "Machine Learning": true,
    "PowerBI": false
  },
  "matching_skills": {
    "Python": true,
    "Machine Learning": true
  }
}


In [ ]:
# Section 3
# File: analysis_prompt_builder.py

from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from src.core.data_models.analysis_result_model import JobAnalysisResult
from typing import List

class AnalysisPromptBuilder:
    """
    Purpose:
        Builds the structured prompt for the LLM to analyze a job vacancy.

    Capabilities:
        - Constructs a system message with instructions.
        - Constructs a human message with the actual job description and skills.
        - Ensures output format matches JobAnalysisResult schema.

    Reasoning:
        Centralizing prompt logic ensures consistency across the agent flow.
    """

    def __init__(self) -> None:
        self.parser = PydanticOutputParser(pydantic_object=JobAnalysisResult)

    def build_prompt(self, skillsets: List[str], job_to_apply: str) -> ChatPromptTemplate:
        """
        Builds a ChatPromptTemplate ready to be sent to the LLM.

        Args:
            skillsets: List of user's skills fetched from service_cover_letter.
            job_to_apply: Raw job vacancy text.

        Returns:
            ChatPromptTemplate: Fully assembled LLM prompt.
        """
        system_analysis_template_str = """
        You are a senior HR analyst working for a career advisory platform.
        Your job is to analyze the job vacancy provided and extract structured insights.
        Ensure your response strictly follows this format:

        {format_instructions}
        """

        system_prompt = SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                template=system_analysis_template_str,
                input_variables=[],
                partial_variables={"format_instructions": self.parser.get_format_instructions()}
            )
        )

        human_analysis_template_str = """
        Job Vacancy Description:
        {job_position}

        Candidate's Skills:
        {my_skills}
        """

        human_prompt = HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                template=human_analysis_template_str,
                input_variables=["job_position", "my_skills"]
            )
        )

        return ChatPromptTemplate(messages=[system_prompt, human_prompt])
    
# Instantiate & test in notebook

# Job description (normally from frontend form)
job_description = """
We are seeking a Data Scientist at InnovativeAI.
The role requires Python, SQL, and Machine Learning expertise.
"""

# Instantiate & build the prompt
prompt_builder = AnalysisPromptBuilder()
prompt = prompt_builder.build_prompt(skillsets, job_description)

# Render to see what actually gets sent to LLM
formatted_prompt = prompt.format_messages(job_position=job_description, my_skills=skillsets)

print("\n✅ Final Analysis Prompt (ready for LLM):")
for message in formatted_prompt:
    print(f"\n[{message.type.upper()}]\n{message.content}")




✅ Final Analysis Prompt (ready for LLM):

[SYSTEM]

        You are a senior HR analyst working for a career advisory platform.
        Your job is to analyze the job vacancy provided and extract structured insights.
        Ensure your response strictly follows this format:

        The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"company_name": {"description": "Identified company name", "title": "Company Name", "type": "string"}, "job_title": {"description": "Identified job title", "title": "Job Title", "type": "string"}, "analysis_output": {"description": "Analysis of th

In [ ]:
# Section 4
# backend/aiml_models/agent_teams/agent_tailored_cover_letter/src/core/company_analysis/components/analysis_rules_validator.py
from typing import List, Dict, Any
from src.core.data_models.analysis_result_model import JobAnalysisResult

class AnalysisRulesValidator:
    """
    Purpose:
        Validates the analysis result against forbidden words and sentences across multiple iterations.
        Supports progressive error tracking for self-correction.

    Capabilities:
        - Scans the analysis output for forbidden content.
        - Tracks all found issues per iteration.
        - Generates reflection instructions for self-correction.
        - Returns structured result instead of raising exceptions.

    Reasoning:
        This separates detection from logic flow.
        ✅ Easy to pass feedback into reflection loops.
        ✅ Supports time-series error analysis (how errors evolve over iterations).
    """

    def validate(self, analysis_result: JobAnalysisResult, forbidden_words: List[str], forbidden_sentences: List[str], iteration: int) -> Dict[str, Any]:
        """
        Validates the analysis output, tracks issues, and prepares correction feedback.

        Args:
            analysis_result: Structured result to validate.
            forbidden_words: List of forbidden words.
            forbidden_sentences: List of forbidden sentences.
            iteration: Current iteration number (for logging).

        Returns:
            Dict containing status, found issues, and reflection feedback.
        """
        issues = self._check_forbidden_content(analysis_result.analysis_output, forbidden_words, forbidden_sentences)

        if not issues["words"] and not issues["sentences"]:
            return {
                "status": "passed",
                "iteration": iteration,
                "found_issues": issues,
                "reflection": None
            }

        reflection = self._generate_reflection(issues)

        return {
            "status": "failed",
            "iteration": iteration,
            "found_issues": issues,
            "reflection": reflection
        }

    def _check_forbidden_content(self, text: str, forbidden_words: List[str], forbidden_sentences: List[str]) -> Dict[str, List[str]]:
        """
        Internal content scan, checks against words & sentences.
        """
        text_lower = text.lower()

        found_words = [word for word in forbidden_words if word.lower() in text_lower]
        found_sentences = [sentence for sentence in forbidden_sentences if sentence.lower() in text_lower]

        return {
            "words": found_words,
            "sentences": found_sentences
        }

    def _generate_reflection(self, issues: Dict[str, List[str]]) -> str:
        """
        Generates self-correction reflection instruction.
        """
        reflection = []
        if issues["words"]:
            reflection.append(f"Remove or replace these words: {', '.join(issues['words'])}.")
        if issues["sentences"]:
            reflection.append(f"Rephrase or remove these sentences: {', '.join(issues['sentences'])}.")
        
        reflection.append("Keep the meaning intact but ensure no forbidden language is present.")
        return " ".join(reflection)


from src.core.company_analysis.components.analysis_rules_validator import AnalysisRulesValidator
from src.core.data_models.analysis_result_model import JobAnalysisResult

validator = AnalysisRulesValidator()

mock_result = JobAnalysisResult(
    company_name="AwesomeTech",
    job_title="Data Scientist",
    analysis_output="This role is a perfect fit for someone eager to excel in machine learning.",
    employees_skills_requirement={"Python": True, "SQL": True},
    matching_skills={"Python": True, "SQL": True}
)

forbidden_words = ["perfect", "eager", "excel"]
forbidden_sentences = ["perfect fit for this role"]

# Simulate first run
feedback = validator.validate(mock_result, forbidden_words, forbidden_sentences, iteration=1)
print("\n✅ Validator Feedback:")
print(feedback)



✅ Validator Feedback:
{'status': 'failed', 'iteration': 1, 'found_issues': {'words': ['perfect', 'eager', 'excel'], 'sentences': []}, 'reflection': 'Remove or replace these words: perfect, eager, excel. Keep the meaning intact but ensure no forbidden language is present.'}


In [ ]:
# section 5
# backend/aiml_models/agent_teams/agent_tailored_cover_letter/src/core/company_analysis/agent_service_class_company_analysis.py


from typing import Dict, List
from src.infrastructure.correction_client import CorrectionsClient
from src.core.company_analysis.components.analysis_prompt_builder import AnalysisPromptBuilder
from src.core.company_analysis.components.analysis_respose_parser import JobAnalysisResultParser
from src.core.company_analysis.components.analysis_rules_validator import AnalysisRulesValidator
from src.core.data_models.analysis_result_model import JobAnalysisResult
from src.infrastructure.llm_client import LLMClient

class AgentServiceClassCompanyAnalysis:
    """
    Purpose:
        Orchestrates the full flow of analyzing a job vacancy using the company analysis agent.

    Capabilities:
        - Fetch forbidden words, sentences, and skills from service_cover_letter.
        - Builds and sends a structured prompt to the LLM via LLMClient.
        - Parses the LLM response into structured data.
        - Validates the result against forbidden content rules.
        - Supports iterative self-correction if validation fails.

    Reasoning:
        This centralizes all flow logic into a single point, maintaining separation between:
        - Infrastructure (API calls, LLM calls)
        - Core logic (prompt, parsing, validation)
    """

    def __init__(
        self,
        corrections_client: CorrectionsClient,
        prompt_builder: AnalysisPromptBuilder,
        response_parser: JobAnalysisResultParser,
        rules_validator: AnalysisRulesValidator,
        llm_client: LLMClient  # This MUST be present
        ) -> None:
        self.corrections_client = corrections_client
        self.prompt_builder = prompt_builder
        self.response_parser = response_parser
        self.rules_validator = rules_validator
        self.llm_client = llm_client
        self.correction_history: List[Dict] = []  # Tracks all corrections across iterations

    def analyze_job_vacancy(self, job_description: str) -> JobAnalysisResult:
        # Fetch forbidden words, sentences, and skills
        forbidden_words = [item["text"] for item in self.corrections_client.fetch_corrections("word")]
        forbidden_sentences = [item["text"] for item in self.corrections_client.fetch_corrections("sentence")]
        skillsets = [item["text"] for item in self.corrections_client.fetch_corrections("skill")]

        # Main generation loop with up to 12 self-correction attempts
        for iteration in range(1, 13):
            prompt = self.prompt_builder.build_prompt(skillsets, job_description)
            formatted_prompt = prompt.format_messages(job_position=job_description, my_skills=skillsets)

            # 🔥 Invoke the real LLM via Ollama
            raw_response = self.llm_client.invoke(formatted_prompt)

            # Parse response
            parsed_response = self.response_parser.parse(raw_response)

            # Validate response
            feedback = self.rules_validator.validate(parsed_response, forbidden_words, forbidden_sentences, iteration)

            if feedback["status"] == "passed":
                print(f"✅ Analysis passed after {iteration} iterations.")
                return parsed_response

            # Store feedback for future self-correction
            self.correction_history.append(feedback)

            # Prepare for next round with feedback (could modify prompt, but for now we log and retry as-is)
            self._log_feedback(feedback)

        raise ValueError("🚨 Analysis failed after 12 correction attempts.")

    def _log_feedback(self, feedback: Dict) -> None:
        print(f"\n❌ Iteration {feedback['iteration']} failed — Reflection for self-correction:\n{feedback['reflection']}\n")




In [ ]:
# Section 6
# File: backend/aiml_models/agent_teams/agent_tailored_cover_letter/src/agents_cover_letter_main.py

# Full End-to-End Test - Notebook Section

from src.infrastructure.correction_client import CorrectionsClient
from src.infrastructure.llm_client import LLMClient
from src.core.company_analysis.components.analysis_prompt_builder import AnalysisPromptBuilder
from src.core.company_analysis.components.analysis_respose_parser import JobAnalysisResultParser
from src.core.company_analysis.components.analysis_rules_validator import AnalysisRulesValidator
from src.core.company_analysis.agent_service_class_company_analysis import AgentServiceClassCompanyAnalysis

# Initialize dependencies
corrections_client = CorrectionsClient()
# llm_client = LLMClient(model="deepseek")  # Adjust if using different model

# Instantiate the full agent
agent = AgentServiceClassCompanyAnalysis(
    corrections_client=corrections_client,
    prompt_builder=AnalysisPromptBuilder(),
    response_parser=JobAnalysisResultParser(),
    rules_validator=AnalysisRulesValidator(),
    llm_client=LLMClient(model="ollama run deepseek-r1:7b")
)

# Example job description (this would normally come from frontend input)
job_description = """
We are looking for a Data Scientist to join our team.
The ideal candidate should have experience in Python, SQL, and Machine Learning.
"""

print()
# Trigger the full analysis process
try:
    final_result = agent.analyze_job_vacancy(job_description)
    print("\n✅ Final Analysis Result (JobAnalysisResult):")
    print(final_result.model_dump_json(indent=2))
except Exception as e:
    print(f"\n🚨 Analysis Process Failed: {e}")



🚨 Analysis Process Failed: Client error '404 Not Found' for url 'http://localhost:11434/api/generate'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404
